<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/Logistic_Regression_%2B_random_oversampling%5BLB_0_66%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google.drive and copy dataset to colab linux instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_DIR ='/content/drive/MyDrive/kaggle/fsnlp_hse/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

WORK_DIR='/content/'
EXPERIMENT='logreg_ro'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

In [ ]:
!mkdir {EXPERIMENT_DIR}

# Install and set up Kaggle API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp
!unzip hse-nlp-bootcamp.zip

 70% 42.0M/59.8M [00:00<00:00, 67.4MB/s]
100% 59.8M/59.8M [00:00<00:00, 72.7MB/s]
Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pprint as pp
import os


# import re
# from pymorphy2 import MorphAnalyzer
# from functools import lru_cache
# from nltk.corpus import stopwords

# from multiprocessing import Pool
# from tqdm import tqdm

import nltk
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from joblib import dump, load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Configuration

In [ ]:
class Config:
  seed = 12536271
  samples_num = 138694
  experiment = EXPERIMENT
  test_size = 0.1

# Main pipeline starts here

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(Config.seed)

## Load and process train data

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['bank_id'] = df['bank'].apply(lambda x: banks_list.index(x))
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df.head()

,bank,feeds,grades,date,bank_id
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3


In [ ]:
len(df)

51476

In [ ]:
s_nums = [0] * 5
class_samples = [None] * 5
class_extra_samples = [None] * 5

In [ ]:
for i in range(5):
  class_samples[i] = df[df['grades']==i+1].copy()
  s_nums[i] = 27739 - len(class_samples[i])
  class_extra_samples[i] = class_samples[i].sample(s_nums[i], random_state=Config.seed, replace=True)

In [ ]:
extra_df = pd.concat([df] + [class_extra_samples[i] for i in range(1,5)]).reset_index(drop=True)

In [ ]:
len(extra_df)

138695

In [ ]:
extra_df.tail(10)

,bank,feeds,grades,date,bank_id
138685,avangard,"Пожалуй, пора уже написать положительный отзыв...",5,2009-12-13 23:07:00,44
138686,vtb,После работы обратилась в офис ВТБ -на Ленина...,5,2020-10-03 18:15:00,3
138687,homecreditbank,"Добрый вечер, отзыв мой будет посвещен сервису...",5,2019-08-21 21:46:00,7
138688,sovcombank,Добрый день! Хочу выразить большую благодарнос...,5,2018-08-07 11:21:00,9
138689,modulbank,Заметил ошибку на карте практически перед самы...,5,2019-08-13 15:52:00,48
138690,tcs,Из минусов - на текущий момент не со всеми бан...,5,2020-12-27 21:38:00,11
138691,homecreditbank,09 июня оформляла кредит в отделении банка Хоу...,5,2020-06-23 15:14:00,7
138692,tcs,"Спасибо сотрудникам Тинькофф банк, которые в о...",5,2019-08-26 18:12:00,11
138693,vtb,Хочу поблагодарить за человеческое отношение к...,5,2016-08-17 10:04:00,3
138694,vtb,Хочу поблагодарить за человеческое отношение к...,5,2016-08-17 10:04:00,3


In [ ]:
for i in range(5):
  print(i+1, len(extra_df[extra_df['grades']==i+1]))

1 27739
2 27739
3 27739
4 27739
5 27739


In [ ]:
subset = extra_df.sample(Config.samples_num, random_state=Config.seed)
subset['text'] = subset.feeds.apply(lambda x: x.lower())
subset.reset_index(drop=True, inplace=True)

## Split data on train and test data

In [ ]:
if Config.test_size > 0.0:
  x_train, x_test, y_train, y_test = train_test_split(subset[['bank_id','text']], subset['grades'], test_size=Config.test_size, random_state=Config.seed)
else:
  x_train = subset[['bank_id','text']]
  y_train = subset['grades']
  x_test = None
  y_test = None

In [ ]:
len(x_train)

124824

In [ ]:
for i in range(5):
  print(i+1, len(y_train[y_train==i+1]))

1 24927
2 24964
3 25006
4 24929
5 24998


## Set up train pipeline

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize))
            ]))
         ])),
    ('clf', LogisticRegression(random_state=Config.seed,  warm_start=True, max_iter=1000))
])

In [ ]:
pp.pprint(sorted(pipeline.get_params().keys()))

In [ ]:
param_grid = {'features__text_features__vec__max_df': [1.0],
              'features__text_features__vec__min_df': [0.001]}

kfold = StratifiedKFold(n_splits=5)
scoring = {'F1': 'f1_micro'}
refit = 'F1'

## Perform GridSearch

In [ ]:
print(f'samples: {Config.samples_num}')
lr_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=10)
lr_model.fit(x_train, y_train)
lr_best = lr_model.best_estimator_

samples: 138694
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Test best model

In [ ]:
if Config.test_size > 0.0:
  pred = lr_best.predict(x_test)
  print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.74      0.67      0.70      2812
           2       0.75      0.73      0.74      2774
           3       0.82      0.92      0.86      2733
           4       0.90      0.96      0.93      2810
           5       0.96      0.90      0.93      2741

    accuracy                           0.84     13870
   macro avg       0.83      0.84      0.83     13870
weighted avg       0.83      0.84      0.83     13870



In [ ]:
if Config.test_size > 0.0:
  print(f1_score(y_test, pred, average='micro'))

0.8350396539293439


In [ ]:
lr_model.best_params_

{'features__text_features__vec__max_df': 1.0,
 'features__text_features__vec__min_df': 0.001}

In [ ]:
lr_model.cv_results_

{'mean_fit_time': array([1069.74394455]),
 'std_fit_time': array([217.6106604]),
 'mean_score_time': array([86.89809303]),
 'std_score_time': array([15.16943573]),
 'param_features__text_features__vec__max_df': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__min_df': masked_array(data=[0.001],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'features__text_features__vec__max_df': 1.0,
   'features__text_features__vec__min_df': 0.001}],
 'split0_test_F1': array([0.82823954]),
 'split1_test_F1': array([0.83512918]),
 'split2_test_F1': array([0.84153815]),
 'split3_test_F1': array([0.85679952]),
 'split4_test_F1': array([0.83996956]),
 'mean_test_F1': array([0.84033519]),
 'std_test_F1': array([0.00944524]),
 'rank_test_F1': array([1], dtype=int32),
 'split0_train_F1': array([0.87711673]),
 'split1_train_F1': array([0.88401646]),
 'split2_train_F1': a

In [ ]:
lr_model.best_score_

0.8403351890210787

In [ ]:
cv_tag = f'[cv_{lr_model.best_score_:0.4f}]'

In [ ]:
cv_tag

'[cv_0.8403]'

## Inference (calculating submission) 

In [ ]:
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
test['text'] = test.feeds.apply(lambda x: x.lower())
test['bank_id'] = test['bank'].apply(lambda x: banks_list.index(x))
test = test[['bank_id', 'text']]
test_pred = lr_best.predict(test)
probs = lr_best.predict_proba(test)
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})
sub

,inds,grades
0,0,2
1,1,1
2,2,1
3,3,1
4,4,3
...,...,...
17215,17215,3
17216,17216,1
17217,17217,1
17218,17218,1


# Save results to files

In [ ]:
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'
model_filename = 'model_'+suffix+'.joblib'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs)
dump(lr_best, model_filename) 

['model_logreg_ro_[sn_138694]_[sd_12536271][cv_0.8403].joblib']

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}
!cp {model_filename} {EXPERIMENT_DIR}

# Submit predictions

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:04<00:00, 29.6kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

# Submit CV ~ 0.79 - best public LB 0.79651